In [1]:
import time
import re
from bs4 import BeautifulSoup #use beautifulsoup to scrape data
from selenium import webdriver


url = "http://millercenter.org/the-presidency/presidential-speeches"

browser = webdriver.Chrome() #change this line depending on your system
browser.get(url) #go to the website 


#This is what we will change if we want the whole data. Right now it only runs for 44th president

for i in range(43,44):
    x = i
    absolutepath_president = f'/html/body/div[2]/div/main/div[2]/div/div[2]/article/div/div[2]/div/div/div/form/div[3]/fieldset/div/div/div/div[{x}]/label'
    president1 = browser.find_element_by_xpath(absolutepath_president)
    president1.click()
    time.sleep(2)


# president = browser.find_element_by_xpath(
#     '/html/body/div[2]/div/main/div[2]/div/div[2]/article/div/div[2]/div/div/div/form/div[3]/fieldset/div/div/div/div[42]/label')
# president.click()





SCROLL_PAUSE_TIME = 0.8


#slight changes to be made!!

# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

soup = BeautifulSoup(browser.page_source, 'lxml')    

In [32]:
import urllib3 
import pandas as pd
from datetime import datetime
import time
import string

link_str = []

for link in soup.findAll('a', href=True):
    if link != None:
        link_str.append(link['href'])

# now I want the link strings containting /the-presidency/
new_link_str = []

for element in link_str:
    if element.find("/presidential-speeches/") >= 0: 
        new_link_str.append(element)
        




http = urllib3.PoolManager()
list2=[]
for element in new_link_str:
    url = "http://millercenter.org" + element
#     print(element)
    response = http.request('GET', url)
    soup_2 = BeautifulSoup(response.data, 'lxml')
    list2.append(soup_2.findAll('div', {"class":"transcript-inner"}))


#converting into string due to weird beautifulsoup output    
list3=[]
for i in range(len(list2)):
    list3.append(str(list2[i]))

# print(list3[1])

#this will be used to clean the text from html tags
def tag_cleanr(text):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', text)
    return cleantext

cleaned_speeches = []

#these replace lines could and should be replaced with regex since we covered that in the syllabus more extensively

for i in range(len(list3)):
    x = tag_cleanr(list3[i])
    x = x.replace('\nTranscript\n', '').replace('\r\n', '')
    cleaned_speeches.append(x)


#Get all the speech dates into a list

pattern = r'(\w+-\d+-\d{4})'
speech_dates=[]


for i in range(len(new_link_str)):
    match = re.findall(pattern, new_link_str[i])
    speech_dates.append(str(match))


for i in range(len(speech_dates)):
    #this feels like its terrible syntax but not sure if there is a nicer way, maybe I will replace it with regex
    
    speech_dates[i] = speech_dates[i].replace('[', '').replace(']', ''.replace("'", "")).replace("'", "")
    speech_dates[i] = datetime.strptime(speech_dates[i], '%B-%d-%Y')


df = pd.DataFrame({'dates':speech_dates, 'speeches':cleaned_speeches})

print(df)




        dates                                           speeches
0  2016-05-15  [Hello Rutgers!  (Applause.)  R-U rah-rah!  (A...
1  2016-03-22  [Thank you.  (Applause.)  Muchas gracias.  Tha...
2  2016-01-12  [Mr. Speaker, Mr. Vice President, Members of C...
3  2015-06-26  [THE PRESIDENT:  Giving all praise and honor t...
4  2015-03-07  [UDIENCE MEMBER:  We love you, President Obama...
5  2015-01-20  [Mr. Speaker, Mr. Vice President, Members of C...
6  2014-11-20  [My fellow Americans, tonight, I’d like to tal...
7  2014-01-28  [Mr. Speaker, Mr. Vice President, Members of C...
8  2013-12-04  [Thank you.  (Applause.)  Thank you, everybody...
9  2013-09-10  [My fellow Americans, tonight I want to talk t...
10 2013-07-24  [Hello, Warrensburg!  (Applause.)  Hello, Mule...
11 2013-07-19  [I wanted to come out here, first of all, to t...
12 2013-04-08  [Hello, Connecticut.  (Applause.)  Thank you. ...
13 2013-03-21  [Thank you.  (Applause.)  Thank you so much. W...
14 2013-03-01  [Good morn